# Train a Linear Regression Model with Watson Machine Learning 

Notebook created by Zeming Zhao on June, 2021

In this notebook, you will learn how to use the Watson Machine Learning Accelerator (WML-A) API and accelerate the processing of Linear Regression model on GPU with Watson Machine Learning Accelerator.

Linear Regression is a simple machine learning model where the response y is modelled by a linear combination of the predictors in X.

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames as the input.

This notebook covers the following sections:

1. [Setup Linear Regression using cuML](#rbm-model)<br>

1. [Training the model on GPU with Watson Machine Learning Accelerator](#gpu)<br>

1. [Setup Linear Regression using sklearning](#rbm-model)<br>

1. [Training the model on CPU with Watson Machine Learning Accelerator](#cpu)<br>

<a id = "rbm-model"></a>
## Step 1 : Setup Linear Regression model using cuML

### Prepare directory and file for writing Linear Regression engine.

In [1]:
from pathlib import Path
model_dir = f'/data/models' 
model_main = f'lr_main.py'
Path(model_dir).mkdir(exist_ok=True)
print("create model directory done.")

create model directory done.


### create a Linear Regression Model based on cuML on GPU

In [2]:
%%writefile {model_dir}/{model_main}

import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression
import datetime
import os

# specify the cache location to /gpfy since ~/.cache is not available
os.environ["CUPY_CACHE_DIR"]="/gpfs/mydatafs/models/"

# Define Parameters for a large regression
n_samples = 2**20 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 399

# keep consistent with diabetes dataset
#n_samples = 442 
#n_features = 10

random_state = 23

# Generate Data
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)

X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)


#  # Copy dataset from GPU memory to host memory.
#  # This is done to later compare CPU and GPU results.
#  X_train = X_cudf.to_pandas()
#  X_test = X_cudf_test.to_pandas()
#  y_train = y_cudf.to_pandas()
#  y_test = y_cudf_test.to_panda
#  
#  # Scikit-learn Model
#  # Fit, predict and evaluate
#  
#  ols_sk = skLinearRegression(fit_intercept=True,normalize=True,n_jobs=-1)
#  ols_sk.fit(X_train, y_train)
#  
#  
#  predict_sk = ols_sk.predict(X_test)
#  
#  r2_score_sk = r2_score(y_cudf_test, predict_sk)

# cuML Model
ols_cuml = cuLinearRegression(fit_intercept=True,normalize=True,algorithm='eig')

# Fit
start = datetime.datetime.now()
ols_cuml.fit(X_cudf, y_cudf)
end = datetime.datetime.now()
print ("train timecost: %.2gs" % ((end-start).total_seconds()))

# Predict
start = datetime.datetime.now()
predict_cuml = ols_cuml.predict(X_cudf_test)
end = datetime.datetime.now()
print ("predict timecost: %.2gs" % ((end-start).total_seconds()))

# Evaluate
start = datetime.datetime.now()
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)
end = datetime.datetime.now()
print ("evaluate timecost: %.2gs" % ((end-start).total_seconds()))

#  print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

Overwriting /data/models/lr_main.py


## Step 2 :  Training the model on GPU with Watson Machine Learning Accelerator

<a id = "gpu"></a>
#### Prepare the model lib for running on GPU:

In [3]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib

Populating the interactive namespace from numpy and matplotlib


#### Configuring your environment and project details
To set up your project details, provide your credentials in this cell. You must include your cluster URL, username, and password.

In [4]:
hostname='wmla-console-wmla.apps.wml1x180.ma.platformlab.ibm.com'  # please enter Watson Machine Learning Accelerator host name
# login='username:password' # please enter the login and password
login='admin:p7PMrMMknVQzEb3ptyj0D6XRTO5PQjYL'
es = base64.b64encode(login.encode('utf-8')).decode("utf-8")
# print(es)
commonHeaders={'Authorization': 'Basic '+es}
req = requests.Session()
auth_url = 'https://{}/auth/v1/logon'.format(hostname)
print(auth_url)

a=requests.get(auth_url,headers=commonHeaders, verify=False)
access_token=a.json()['accessToken']
# print("Access_token: ", access_token)

https://wmla-console-wmla.apps.wml1x180.ma.platformlab.ibm.com/auth/v1/logon


In [5]:
dl_rest_url = 'https://{}/platform/rest/deeplearning/v1'.format(hostname)
commonHeaders={'accept': 'application/json', 'X-Auth-Token': access_token}
req = requests.Session()

In [6]:
# Health check
confUrl = 'https://{}/platform/rest/deeplearning/v1/conf'.format(hostname)
r = req.get(confUrl, headers=commonHeaders, verify=False)


In [14]:
#### define the status checking fuction

In [11]:
import tarfile
import tempfile
import os
import json
import pprint
import pandas as pd
from IPython.display import clear_output

def query_job_status(job_id,refresh_rate=3) :

    execURL = dl_rest_url  +'/execs/'+ job_id['id']
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(execURL, headers=commonHeaders, verify=False)
        monitoring = pd.DataFrame(res.json(), index=[0])
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(monitoring)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['PENDING_CRD_SCHEDULER', 'SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
    return res

In [15]:
model_file = model_dir+"/"+model_main
files = {'file': open(model_file , 'rb')}

args = '--exec-start tensorflow --cs-datastore-meta type=fs \
                     --workerDeviceNum 1 \
                     --conda-env-name rapids-21.06  \
                     --model-main /gpfs/mydatafs/models/lr_main.py --workerDeviceType gpu'
                    # --epochs 5 --batch-size 10000 --workerDeviceType gpu'
                    # --model-main '+  "/gpfs/mydatafs/" + model_file + ' --epochs 5 --batch-size 10000'

In [12]:
starttime = datetime.datetime.now()

r = requests.post(dl_rest_url+'/execs?args='+args, files=files,
                  headers=commonHeaders, verify=False)
if not r.ok:
    print('submit job failed: code=%s, %s'%(r.status_code, r.content))
        
job_status = query_job_status(r.json(),refresh_rate=5)

endtime = datetime.datetime.now()

print("\nTraining cost: ", (endtime - starttime).seconds, " seconds.")

Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-190,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --...,wmla-190,admin,FINISHED,wmla-190,https://wmla-mss:9080,wmla,/gpfs/myresultfs/admin/batchworkdir/wmla-190/_submitted_code,SingleNodeTensorflowTrain,2021-07-02T04:38:25Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-190',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta '
          'type=fs                      --workerDeviceNum '
          '1                      --conda-env-name '
          'rapids-21.06                       --model-main '
          '/gpfs/mydatafs/models/lr_main.py --workerDeviceType gpu ',
  'createTime': '2021-07-02T04:38:25Z',
  'creator': 'admin',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-190',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-190',
  'workDir': '/gpfs/myresultfs/admin/batchworkdir/wmla-190/_submitted_code'}

Training cost:  37  seconds.


## Step 3 :  Setup Linear Regression model using SK-Learning
### create a Linear Regression Model based on sk-learning on CPU

In [13]:
%%writefile {model_dir}/{model_main}

# Code source: Jaques Grobler
# License: BSD 3 clause

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

import datetime

# Load and return the diabetes dataset (regression).
# n_sample: 442
# n_feature: 10
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
start = datetime.datetime.now()
regr.fit(diabetes_X_train, diabetes_y_train)
end = datetime.datetime.now()
print ("training timecost: %.2gs" % ((end-start).total_seconds()))

# Make predictions using the testing set
start = datetime.datetime.now()
diabetes_y_pred = regr.predict(diabetes_X_test)
end = datetime.datetime.now()
print ("predict timecost: %.2gs" % ((end-start).total_seconds()))

# The coefficients
print('Coefficients: ', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(diabetes_y_test, diabetes_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(diabetes_y_test, diabetes_y_pred))

# Plot outputs
plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
plt.plot(diabetes_X_test, diabetes_y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

Overwriting /data/models/lr_main.py


## Step 4 :  Training the SK-Learning model on CPU with Watson Machine Learning Accelerator

In [16]:
model_file = model_dir+"/"+model_main
files = {'file': open(model_file , 'rb')}

args = '--exec-start tensorflow --cs-datastore-meta type=fs \
                     --workerDeviceNum 1 \
                     --conda-env-name rapids-21.06  \
                     --model-main /gpfs/mydatafs/models/lr_main.py --workerDeviceType cpu'
                    # --epochs 5 --batch-size 10000 --workerDeviceType gpu'
                    # --model-main '+  "/gpfs/mydatafs/" + model_file + ' --epochs 5 --batch-size 10000'

In [17]:
import datetime

starttime = datetime.datetime.now()

# ! python {model_dir}/{model_main} # --no-cuda --epochs 5 --batch-size 10000
r = requests.post(dl_rest_url+'/execs?args='+args, files=files,
                  headers=commonHeaders, verify=False)
if not r.ok:
    print('submit job failed: code=%s, %s'%(r.status_code, r.content))
        
job_status = query_job_status(r.json(),refresh_rate=5)

endtime = datetime.datetime.now()
print("Training cost: ", (endtime - starttime).seconds, " seconds.")


Refreshing every 5 seconds


,id,args,submissionId,creator,state,appId,schedulerUrl,modelFileOwnerName,workDir,appName,createTime,elastic,nameSpace,numWorker,framework
0,wmla-191,--exec-start tensorflow --cs-datastore-meta type=fs --workerDeviceNum 1 --...,wmla-191,admin,FINISHED,wmla-191,https://wmla-mss:9080,wmla,/gpfs/myresultfs/admin/batchworkdir/wmla-191/_submitted_code,SingleNodeTensorflowTrain,2021-07-02T04:51:59Z,False,wmla,1,tensorflow


{ 'appId': 'wmla-191',
  'appName': 'SingleNodeTensorflowTrain',
  'args': '--exec-start tensorflow --cs-datastore-meta '
          'type=fs                      --workerDeviceNum '
          '1                      --conda-env-name '
          'rapids-21.06                       --model-main '
          '/gpfs/mydatafs/models/lr_main.py --workerDeviceType cpu ',
  'createTime': '2021-07-02T04:51:59Z',
  'creator': 'admin',
  'elastic': False,
  'framework': 'tensorflow',
  'id': 'wmla-191',
  'modelFileOwnerName': 'wmla',
  'nameSpace': 'wmla',
  'numWorker': 1,
  'schedulerUrl': 'https://wmla-mss:9080',
  'state': 'FINISHED',
  'submissionId': 'wmla-191',
  'workDir': '/gpfs/myresultfs/admin/batchworkdir/wmla-191/_submitted_code'}
Training cost:  28  seconds.
